**Author: [Dev Kumar Maan](https://www.linkedin.com/in/dev-kumar-maan-3a6369180/)**

**Institution: National Institute of Technology, Delhi**

**Email: dev.maan02@gmail.com**

# Natural Language Query Agent (Dataset Preparation)

The primary objective of this project is to develop a Natural Language Query Agent that leverages Large Language Models (LLMs) to provide concise responses to straightforward queries within a substantial dataset comprising lecture notes. 

This notebook offers a comprehensive guide to preparing the dataset for use in our final pipeline, facilitating answers to conversational questions.

> The data sources utilized for this project encompass the following:

- [Stanford LLMs Lecture Notes](https://stanford-cs324.github.io/winter2022/lectures/)

- [Awesome LLM Milestone Papers](https://github.com/Hannibal046/Awesome-LLM#milestone-papers)

- [An Extensive Paper List (and Various Resources) on NLP for Social Good](https://github.com/zhijing-jin/NLP4SocialGood_Papers?tab=readme-ov-file)

Let's begin by installing the essential libraries and frameworks required to run this project. The following tools are necessary for its proper functionality.

- [transformers](https://pypi.org/project/transformers/)
- [accelerate](https://pypi.org/project/accelerate/)
- [einops](https://pypi.org/project/einops/)
- [langchain](https://pypi.org/project/langchain/)
- [xformers](https://pypi.org/project/xformers/)
- [bitsandbytes](https://pypi.org/project/bitsandbytes/)
- [faiss-gpu](https://pypi.org/project/faiss-gpu/)
- [sentence_transformers](https://pypi.org/project/sentence-transformers/)
- [pypdf](https://pypi.org/project/pypdf/)


In [15]:
!pip install -qU transformers accelerate einops langchain xformers bitsandbytes faiss-cpu sentence_transformers

In [16]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"

In [28]:
!pip install pypdf

In [21]:
from torch import cuda, bfloat16
import transformers
import torch
import pickle
from transformers import StoppingCriteria, StoppingCriteriaList
from langchain.llms import HuggingFacePipeline
from langchain.document_loaders import WebBaseLoader
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.chains import ConversationalRetrievalChain

USER_AGENT environment variable not set, consider setting it to identify your requests.


#### Now, we'll generate the dataset that will be employed in our project. We will utilize the document loaders provided by Langchain, which include:

- [WebBaseLoader](https://python.langchain.com/docs/integrations/document_loaders/web_base): WebBaseLoader represents a robust framework designed for extracting text content from HTML web pages and transforming it into a document format suitable for a wide range of downstream tasks.
- [PyPDF](https://python.langchain.com/docs/modules/data_connection/document_loaders/pdf): PyPDF is a potent framework crafted for the extraction of text from PDF files.


We will retrieve links to webpages and PDF files from the following .txt documents:

	ema_dataset_web_links.txt
	ema_dataset_pdf_links.txt

In [22]:
def read_file(file_path):
    lines = []
    with open(file_path, 'r') as file:
        for line in file:
            line = line.rstrip('\n')
            lines.append(line)
    return lines

In [23]:
def read_pdf(file_path):
    documents_pdf =[]
    pdf_links = read_file(file_path)
    for link in pdf_links:
        loader = PyPDFLoader(link)
        pdf = loader.load()
        documents_pdf = documents_pdf + pdf
    return documents_pdf

In [25]:
file_path = 'ema_dataset_web_links.txt'
web_links = read_file(file_path)
loader = WebBaseLoader(web_links)
documents_web_links = loader.load()

In [30]:
file_path = 'ema_dataset_pdf_links.txt'
documents_pdf = read_pdf(file_path)

#### At the end of the process, we'll preserve the custom dataset as a .pkl file, an integral part of our pipeline.

In [31]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=20)
all_splits_p0 = text_splitter.split_documents(documents_web_links)
all_splits_p1 = text_splitter.split_documents(documents_pdf)

all_splits = all_splits_p0 + all_splits_p1

with open('ema_dataset.pkl', 'wb') as file:
    pickle.dump(all_splits, file)